In [1]:
import $ivy.`org.apache.spark::spark-sql:3.5.0`
import org.apache.spark.sql._
import org.apache.spark.sql.functions._

import $ivy.$
import org.apache.spark.sql._
import org.apache.spark.sql.functions._

In [2]:
import org.apache.log4j.{Level, Logger}
Logger.getLogger("org").setLevel(Level.OFF)

import org.apache.log4j.{Level, Logger}

In [3]:
val spark = {
  NotebookSparkSession.builder()
    .master("local[*]")
    .getOrCreate()
}

import spark.implicits._

Loading spark-stubs

Creating SparkSession

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
24/09/06 12:47:09 INFO SparkContext: Running Spark version 3.5.0
24/09/06 12:47:09 INFO SparkContext: OS info Linux, 6.8.0-40-generic, amd64
24/09/06 12:47:09 INFO SparkContext: Java version 11.0.24
24/09/06 12:47:09 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/09/06 12:47:10 INFO ResourceUtils: ==============================================================
24/09/06 12:47:10 INFO ResourceUtils: No custom resources configured for spark.driver.
24/09/06 12:47:10 INFO ResourceUtils: ==============================================================
24/09/06 12:47:10 INFO SparkContext: Submitted application: 3a9a0646-b544-4ee7-a632-21dedf6ab5f6
24/09/06 12:47:10 INFO ResourceProfile: Default ResourceProfile created, executor resources: Map(cores -> name: cores, amount: 1, script: , vendor: , memory -> name: memory, amount: 1024, scr

Spark UI

spark: SparkSession = org.apache.spark.sql.SparkSession@7d9263cc
import spark.implicits._

In [4]:
spark.version

res4: String = "3.5.0"

## Create Dataset

### Из коллекции

In [5]:
org.apache.spark.sql.catalyst.encoders.OuterScopes.addOuterScope(this);

case class Person(name: String, age: Long)

val data = Seq(Person("Andy", 32L), Person("Mike", 25L), Person("Piter", 40L))

defined class Person
data: Seq[Person] = List(
  Person("Andy", 32L),
  Person("Mike", 25L),
  Person("Piter", 40L)
)

In [6]:
val ds = data.toDS()
ds.show

24/09/06 12:47:16 INFO SharedState: Setting hive.metastore.warehouse.dir ('null') to the value of spark.sql.warehouse.dir.
24/09/06 12:47:16 INFO SharedState: Warehouse path is 'file:/home/vadim/workspace/Spark/Dataset/spark-warehouse'.
24/09/06 12:47:17 INFO CodeGenerator: Code generated in 223.818055 ms
24/09/06 12:47:18 INFO CodeGenerator: Code generated in 10.679729 ms
24/09/06 12:47:18 INFO CodeGenerator: Code generated in 14.035052 ms


+-----+---+
| name|age|
+-----+---+
| Andy| 32|
| Mike| 25|
|Piter| 40|
+-----+---+



ds: Dataset[Person] = [name: string, age: bigint]

In [7]:
val ds2 = spark.createDataset(data)
ds2.show

+-----+---+
| name|age|
+-----+---+
| Andy| 32|
| Mike| 25|
|Piter| 40|
+-----+---+



ds2: Dataset[Person] = [name: string, age: bigint]

In [8]:
ds.schema

res8: types.StructType = StructType(
  StructField("name", StringType, true, {}),
  StructField("age", LongType, false, {})
)

In [9]:
ds.schema.equals(ds2.schema)

res9: Boolean = true

### Из внешнего источника

In [10]:
val sparkHome = sys.env.get("SPARK_HOME").get
val path = s"$sparkHome/examples/src/main/resources/people.json"
val peopleDF = spark.read.json(path)
val peopleDS = peopleDF.as[Person]

peopleDS.show()

24/09/06 12:47:21 INFO InMemoryFileIndex: It took 42 ms to list leaf files for 1 paths.
24/09/06 12:47:21 INFO InMemoryFileIndex: It took 1 ms to list leaf files for 1 paths.
24/09/06 12:47:21 INFO FileSourceStrategy: Pushed Filters: 
24/09/06 12:47:21 INFO FileSourceStrategy: Post-Scan Filters: 
24/09/06 12:47:21 INFO MemoryStore: Block broadcast_0 stored as values in memory (estimated size 248.4 KiB, free 4.5 GiB)
24/09/06 12:47:21 INFO MemoryStore: Block broadcast_0_piece0 stored as bytes in memory (estimated size 46.7 KiB, free 4.5 GiB)
24/09/06 12:47:21 INFO BlockManagerInfo: Added broadcast_0_piece0 in memory on ubuntu:44089 (size: 46.7 KiB, free: 4.5 GiB)
24/09/06 12:47:21 INFO SparkContext: Created broadcast 0 from json at cell10.sc:3
24/09/06 12:47:21 INFO FileSourceScanExec: Planning scan with bin packing, max size: 4194304 bytes, open cost is considered as scanning 4194304 bytes.
24/09/06 12:47:21 INFO SparkContext: Starting job: json at cell10.sc:3
24/09/06 12:47:21 INFO DA

24/09/06 12:47:21 INFO MemoryStore: Block broadcast_1 stored as values in memory (estimated size 65.0 KiB, free 4.5 GiB)
24/09/06 12:47:21 INFO MemoryStore: Block broadcast_1_piece0 stored as bytes in memory (estimated size 19.4 KiB, free 4.5 GiB)
24/09/06 12:47:21 INFO BlockManagerInfo: Added broadcast_1_piece0 in memory on ubuntu:44089 (size: 19.4 KiB, free: 4.5 GiB)
24/09/06 12:47:21 INFO SparkContext: Created broadcast 1 from broadcast at DAGScheduler.scala:1580
24/09/06 12:47:21 INFO DAGScheduler: Submitting 1 missing tasks from ResultStage 0 (MapPartitionsRDD[3] at json at cell10.sc:3) (first 15 tasks are for partitions Vector(0))
24/09/06 12:47:21 INFO TaskSchedulerImpl: Adding task set 0.0 with 1 tasks resource profile 0
24/09/06 12:47:21 INFO TaskSetManager: Starting task 0.0 in stage 0.0 (TID 0) (ubuntu, executor driver, partition 0, PROCESS_LOCAL, 27039 bytes) 
24/09/06 12:47:21 INFO Executor: Running task 0.0 in stage 0.0 (TID 0)
24/09/06 12:47:21 INFO FileScanRDD: Reading 

24/09/06 12:47:22 INFO MemoryStore: Block broadcast_3 stored as values in memory (estimated size 14.7 KiB, free 4.5 GiB)
24/09/06 12:47:22 INFO MemoryStore: Block broadcast_3_piece0 stored as bytes in memory (estimated size 7.1 KiB, free 4.5 GiB)
24/09/06 12:47:22 INFO BlockManagerInfo: Added broadcast_3_piece0 in memory on ubuntu:44089 (size: 7.1 KiB, free: 4.5 GiB)
24/09/06 12:47:22 INFO SparkContext: Created broadcast 3 from broadcast at DAGScheduler.scala:1580
24/09/06 12:47:22 INFO DAGScheduler: Submitting 1 missing tasks from ResultStage 1 (MapPartitionsRDD[7] at show at cell10.sc:6) (first 15 tasks are for partitions Vector(0))
24/09/06 12:47:22 INFO TaskSchedulerImpl: Adding task set 1.0 with 1 tasks resource profile 0
24/09/06 12:47:22 INFO TaskSetManager: Starting task 0.0 in stage 1.0 (TID 1) (ubuntu, executor driver, partition 0, PROCESS_LOCAL, 27039 bytes) 
24/09/06 12:47:22 INFO Executor: Running task 0.0 in stage 1.0 (TID 1)
24/09/06 12:47:22 INFO CodeGenerator: Code gen

+----+-------+
| age|   name|
+----+-------+
|NULL|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



sparkHome: String = "/opt/spark"
path: String = "/opt/spark/examples/src/main/resources/people.json"
peopleDF: DataFrame = [age: bigint, name: string]
peopleDS: Dataset[Person] = [age: bigint, name: string]

## Основные операции

In [11]:
val names = ds.map(_.name)
names.show

24/09/06 12:47:22 INFO CodeGenerator: Code generated in 16.01657 ms
24/09/06 12:47:22 INFO CodeGenerator: Code generated in 6.843646 ms
24/09/06 12:47:22 INFO SparkContext: Starting job: show at cell11.sc:2
24/09/06 12:47:22 INFO DAGScheduler: Got job 2 (show at cell11.sc:2) with 1 output partitions
24/09/06 12:47:22 INFO DAGScheduler: Final stage: ResultStage 2 (show at cell11.sc:2)
24/09/06 12:47:22 INFO DAGScheduler: Parents of final stage: List()
24/09/06 12:47:22 INFO DAGScheduler: Missing parents: List()
24/09/06 12:47:22 INFO DAGScheduler: Submitting ResultStage 2 (MapPartitionsRDD[10] at show at cell11.sc:2), which has no missing parents


24/09/06 12:47:22 INFO MemoryStore: Block broadcast_4 stored as values in memory (estimated size 11.9 KiB, free 4.5 GiB)
24/09/06 12:47:22 INFO MemoryStore: Block broadcast_4_piece0 stored as bytes in memory (estimated size 5.1 KiB, free 4.5 GiB)
24/09/06 12:47:22 INFO BlockManagerInfo: Added broadcast_4_piece0 in memory on ubuntu:44089 (size: 5.1 KiB, free: 4.5 GiB)
24/09/06 12:47:22 INFO SparkContext: Created broadcast 4 from broadcast at DAGScheduler.scala:1580
24/09/06 12:47:22 INFO DAGScheduler: Submitting 1 missing tasks from ResultStage 2 (MapPartitionsRDD[10] at show at cell11.sc:2) (first 15 tasks are for partitions Vector(0))
24/09/06 12:47:22 INFO TaskSchedulerImpl: Adding task set 2.0 with 1 tasks resource profile 0
24/09/06 12:47:22 INFO TaskSetManager: Starting task 0.0 in stage 2.0 (TID 2) (ubuntu, executor driver, partition 0, PROCESS_LOCAL, 26607 bytes) 
24/09/06 12:47:22 INFO Executor: Running task 0.0 in stage 2.0 (TID 2)
24/09/06 12:47:22 INFO CodeGenerator: Code ge

24/09/06 12:47:23 INFO MemoryStore: Block broadcast_5_piece0 stored as bytes in memory (estimated size 5.1 KiB, free 4.5 GiB)
24/09/06 12:47:23 INFO BlockManagerInfo: Added broadcast_5_piece0 in memory on ubuntu:44089 (size: 5.1 KiB, free: 4.5 GiB)
24/09/06 12:47:23 INFO SparkContext: Created broadcast 5 from broadcast at DAGScheduler.scala:1580
24/09/06 12:47:23 INFO DAGScheduler: Submitting 2 missing tasks from ResultStage 3 (MapPartitionsRDD[10] at show at cell11.sc:2) (first 15 tasks are for partitions Vector(1, 2))
24/09/06 12:47:23 INFO TaskSchedulerImpl: Adding task set 3.0 with 2 tasks resource profile 0
24/09/06 12:47:23 INFO TaskSetManager: Starting task 0.0 in stage 3.0 (TID 3) (ubuntu, executor driver, partition 1, PROCESS_LOCAL, 26607 bytes) 
24/09/06 12:47:23 INFO TaskSetManager: Starting task 1.0 in stage 3.0 (TID 4) (ubuntu, executor driver, partition 2, PROCESS_LOCAL, 26607 bytes) 
24/09/06 12:47:23 INFO Executor: Running task 0.0 in stage 3.0 (TID 3)
24/09/06 12:47:23

+-----+
|value|
+-----+
| Andy|
| Mike|
|Piter|
+-----+



names: Dataset[String] = [value: string]

In [12]:
val age30 = ds.filter(_.age > 30)
age30.show

24/09/06 12:47:23 INFO BlockManagerInfo: Removed broadcast_4_piece0 on ubuntu:44089 in memory (size: 5.1 KiB, free: 4.5 GiB)
24/09/06 12:47:23 INFO BlockManagerInfo: Removed broadcast_3_piece0 on ubuntu:44089 in memory (size: 7.1 KiB, free: 4.5 GiB)
24/09/06 12:47:23 INFO BlockManagerInfo: Removed broadcast_2_piece0 on ubuntu:44089 in memory (size: 46.7 KiB, free: 4.5 GiB)
24/09/06 12:47:23 INFO BlockManagerInfo: Removed broadcast_0_piece0 on ubuntu:44089 in memory (size: 46.7 KiB, free: 4.5 GiB)
24/09/06 12:47:23 INFO BlockManagerInfo: Removed broadcast_5_piece0 on ubuntu:44089 in memory (size: 5.1 KiB, free: 4.5 GiB)
24/09/06 12:47:23 INFO BlockManagerInfo: Removed broadcast_1_piece0 on ubuntu:44089 in memory (size: 19.4 KiB, free: 4.5 GiB)
24/09/06 12:47:23 INFO CodeGenerator: Code generated in 14.076184 ms
24/09/06 12:47:23 INFO SparkContext: Starting job: show at cell12.sc:2
24/09/06 12:47:23 INFO DAGScheduler: Got job 4 (show at cell12.sc:2) with 1 output partitions
24/09/06 12:4

24/09/06 12:47:23 INFO MemoryStore: Block broadcast_6 stored as values in memory (estimated size 10.1 KiB, free 4.5 GiB)
24/09/06 12:47:23 INFO MemoryStore: Block broadcast_6_piece0 stored as bytes in memory (estimated size 4.8 KiB, free 4.5 GiB)
24/09/06 12:47:23 INFO BlockManagerInfo: Added broadcast_6_piece0 in memory on ubuntu:44089 (size: 4.8 KiB, free: 4.5 GiB)
24/09/06 12:47:23 INFO SparkContext: Created broadcast 6 from broadcast at DAGScheduler.scala:1580
24/09/06 12:47:23 INFO DAGScheduler: Submitting 1 missing tasks from ResultStage 4 (MapPartitionsRDD[13] at show at cell12.sc:2) (first 15 tasks are for partitions Vector(0))
24/09/06 12:47:23 INFO TaskSchedulerImpl: Adding task set 4.0 with 1 tasks resource profile 0
24/09/06 12:47:23 INFO TaskSetManager: Starting task 0.0 in stage 4.0 (TID 5) (ubuntu, executor driver, partition 0, PROCESS_LOCAL, 26607 bytes) 
24/09/06 12:47:23 INFO Executor: Running task 0.0 in stage 4.0 (TID 5)
24/09/06 12:47:23 INFO CodeGenerator: Code ge

24/09/06 12:47:23 INFO MemoryStore: Block broadcast_7_piece0 stored as bytes in memory (estimated size 4.8 KiB, free 4.5 GiB)
24/09/06 12:47:23 INFO BlockManagerInfo: Added broadcast_7_piece0 in memory on ubuntu:44089 (size: 4.8 KiB, free: 4.5 GiB)
24/09/06 12:47:23 INFO SparkContext: Created broadcast 7 from broadcast at DAGScheduler.scala:1580
24/09/06 12:47:23 INFO DAGScheduler: Submitting 2 missing tasks from ResultStage 5 (MapPartitionsRDD[13] at show at cell12.sc:2) (first 15 tasks are for partitions Vector(1, 2))
24/09/06 12:47:23 INFO TaskSchedulerImpl: Adding task set 5.0 with 2 tasks resource profile 0
24/09/06 12:47:23 INFO TaskSetManager: Starting task 0.0 in stage 5.0 (TID 6) (ubuntu, executor driver, partition 1, PROCESS_LOCAL, 26607 bytes) 
24/09/06 12:47:23 INFO TaskSetManager: Starting task 1.0 in stage 5.0 (TID 7) (ubuntu, executor driver, partition 2, PROCESS_LOCAL, 26607 bytes) 
24/09/06 12:47:23 INFO Executor: Running task 1.0 in stage 5.0 (TID 7)
24/09/06 12:47:23

+-----+---+
| name|age|
+-----+---+
| Andy| 32|
|Piter| 40|
+-----+---+



age30: Dataset[Person] = [name: string, age: bigint]

In [13]:
val df1 = ds.select(col("name"), expr("age + 1"))
df1.show

+-----+---------+
| name|(age + 1)|
+-----+---------+
| Andy|       33|
| Mike|       26|
|Piter|       41|
+-----+---------+



df1: DataFrame = [name: string, (age + 1): bigint]

In [14]:
val jsonDS = ds.toJSON
jsonDS.show(false)

24/09/06 12:47:24 INFO CodeGenerator: Code generated in 10.594137 ms
24/09/06 12:47:24 INFO SparkContext: Starting job: show at cell14.sc:2
24/09/06 12:47:24 INFO DAGScheduler: Got job 6 (show at cell14.sc:2) with 1 output partitions
24/09/06 12:47:24 INFO DAGScheduler: Final stage: ResultStage 6 (show at cell14.sc:2)
24/09/06 12:47:24 INFO DAGScheduler: Parents of final stage: List()
24/09/06 12:47:24 INFO DAGScheduler: Missing parents: List()
24/09/06 12:47:24 INFO DAGScheduler: Submitting ResultStage 6 (MapPartitionsRDD[19] at show at cell14.sc:2), which has no missing parents


24/09/06 12:47:24 INFO MemoryStore: Block broadcast_8 stored as values in memory (estimated size 30.3 KiB, free 4.5 GiB)
24/09/06 12:47:24 INFO MemoryStore: Block broadcast_8_piece0 stored as bytes in memory (estimated size 12.7 KiB, free 4.5 GiB)
24/09/06 12:47:24 INFO BlockManagerInfo: Added broadcast_8_piece0 in memory on ubuntu:44089 (size: 12.7 KiB, free: 4.5 GiB)
24/09/06 12:47:24 INFO SparkContext: Created broadcast 8 from broadcast at DAGScheduler.scala:1580
24/09/06 12:47:24 INFO DAGScheduler: Submitting 1 missing tasks from ResultStage 6 (MapPartitionsRDD[19] at show at cell14.sc:2) (first 15 tasks are for partitions Vector(0))
24/09/06 12:47:24 INFO TaskSchedulerImpl: Adding task set 6.0 with 1 tasks resource profile 0
24/09/06 12:47:24 INFO TaskSetManager: Starting task 0.0 in stage 6.0 (TID 8) (ubuntu, executor driver, partition 0, PROCESS_LOCAL, 26607 bytes) 
24/09/06 12:47:24 INFO Executor: Running task 0.0 in stage 6.0 (TID 8)
24/09/06 12:47:24 INFO CodeGenerator: Code 

24/09/06 12:47:24 INFO MemoryStore: Block broadcast_9 stored as values in memory (estimated size 30.3 KiB, free 4.5 GiB)
24/09/06 12:47:24 INFO MemoryStore: Block broadcast_9_piece0 stored as bytes in memory (estimated size 12.7 KiB, free 4.5 GiB)
24/09/06 12:47:24 INFO BlockManagerInfo: Added broadcast_9_piece0 in memory on ubuntu:44089 (size: 12.7 KiB, free: 4.5 GiB)
24/09/06 12:47:24 INFO SparkContext: Created broadcast 9 from broadcast at DAGScheduler.scala:1580
24/09/06 12:47:24 INFO DAGScheduler: Submitting 2 missing tasks from ResultStage 7 (MapPartitionsRDD[19] at show at cell14.sc:2) (first 15 tasks are for partitions Vector(1, 2))
24/09/06 12:47:24 INFO TaskSchedulerImpl: Adding task set 7.0 with 2 tasks resource profile 0
24/09/06 12:47:24 INFO TaskSetManager: Starting task 0.0 in stage 7.0 (TID 9) (ubuntu, executor driver, partition 1, PROCESS_LOCAL, 26607 bytes) 
24/09/06 12:47:24 INFO TaskSetManager: Starting task 1.0 in stage 7.0 (TID 10) (ubuntu, executor driver, partit

+-------------------------+
|value                    |
+-------------------------+
|{"name":"Andy","age":32} |
|{"name":"Mike","age":25} |
|{"name":"Piter","age":40}|
+-------------------------+



jsonDS: Dataset[String] = [value: string]